This notebook relies on the following cmd line tools, ensure they are installed and in the system
- tippecanoe
- mapshaper
- aws cli

In [3]:
from pathlib import Path
import sys
import logging

from IPython.lib import backgroundjobs as bg

scripts_dir = Path('..').joinpath('src')
if scripts_dir not in sys.path:
    sys.path.insert(0, scripts_dir.resolve().as_posix())

from pipelines.pipes import get_pipes, execution_order, filter_pipes

logging.basicConfig(level=logging.DEBUG)
logging.getLogger("requests").setLevel(logging.WARNING)
logging.getLogger("urllib3").setLevel(logging.WARNING)
logging.getLogger("fiona").setLevel(logging.WARNING)
jobs = bg.BackgroundJobManager()

TypeError: ExtractParams.__init__() missing 1 required positional argument: 'source'

In [2]:
# def execute_pipes(pipes):
#     ordered_pipes = execution_order(pipes)
#     for pipe in ordered_pipes:
#         jobs.new(pipe().execute)

# # this code will execute all pipelines selected in background.

# mypipes_subset = filter_pipes(get_pipes().items(), ['eez_tiles'])
# execute_pipes(mypipes_subset)

In [2]:
get_pipes()

{'eez_intermediate': pipelines.intermediate_pipes.EEZIntermediatePipe.EEZIntermediatePipe,
 'mpaatlas_intermediate': pipelines.intermediate_pipes.MpaAtlasIntermediatePipe.MpaAtlasIntermediatePipe,
 'mpa_intermediate': pipelines.intermediate_pipes.MpasIntermediatePipe.MpasIntermediatePipe,
 'protectedseas_intermediate': pipelines.intermediate_pipes.ProtectedSeasIntermediatePipe.ProtectedSeasIntermediatePipe,
 'eez_tiles': pipelines.tiles_pipes.EEZTilesPipe.EEZTilesPipe,
 'mpaatlas_tiles': pipelines.tiles_pipes.MpaAtlasTilesPipe.MpaAtlasTilesPipe,
 'mpa_tiles': pipelines.tiles_pipes.MpasTilesPipe.MpasTilesPipe,
 'protectedseas_tiles': pipelines.tiles_pipes.ProtectedSeasTilesPipe.ProtectedSeasTilesPipe,
 'regions_tiles': pipelines.tiles_pipes.RegionsTilesPipe.RegionsTilesPipe,
 'eez_locations_precalc': pipelines.precalc_pipes.LocationsPipe.LocationsPipe,
 'mpa_coverage_precalc': pipelines.precalc_pipes.PACoveragePipe.PACoveragePipe}

In [3]:
mypipes_subset = filter_pipes(get_pipes(), ["eez_locations_precalc"])

In [9]:
for n, pipe in mypipes_subset.items():
    new_pipe = pipe()
    new_pipe.extract().transform()

DEBUG:pipelines.settings:/home/mambauser/data
INFO:pipelines.base_pipe:Pipeline mpa_intermediate running at 2023-11-16 17:37:32.700031: starting extract...
INFO:utils:File /home/mambauser/data/mpa_intermediate/WDPA_WDOECM_Nov2023_Public_marine_shp.zip already exists.
INFO:pipelines.base_pipe:Pipeline mpa_intermediate finish at 2023-11-16 17:37:32.871508: Success executing extract
INFO:pipelines.base_pipe:Pipeline mpa_intermediate running at 2023-11-16 17:37:32.871822: starting transform...
INFO:pipelines.base_pipe:Pipeline mpa_intermediate finish at 2023-11-16 17:37:32.872314: Success executing transform
INFO:pipelines.base_pipe:Pipeline mpa_intermediate running at 2023-11-16 17:37:32.872605: starting load...
DEBUG:google.auth.transport.requests:Making request: POST https://oauth2.googleapis.com/token
INFO:pipelines.base_pipe:Pipeline mpa_intermediate finish at 2023-11-16 17:38:00.909038: Success executing load


In [3]:
#!micromamba install pandera pandera-hypotheses pandera-io pandera-geopandas -c conda-forge -y

In [4]:
from logging import getLogger
from pathlib import Path
from typing import Union
import pandas as pd
import geopandas as gpd
import numpy as np
import json

from pipelines.utils import load_iso_mapping, load_regions

from pipelines.output_schemas import LocationSchema

In [5]:
input_folder = Path("../data/eez_intermediate").absolute()
location_code = pd.read_csv(input_folder.joinpath("locations_code.csv"))
columns_to_keep = ["code", "name", "totalMarineArea", "type", "groups", "bounds", "id"]
rename = {
    "iso": "code",
    "AREA_KM2": "totalMarineArea",
    "location_type": "type",
}

In [12]:
file = gpd.read_file(input_folder.joinpath("eez_intermediate", "eez_intermediate.shp").as_posix())